In [ ]:
import cv2
import numpy as np
from pygame import mixer
from tensorflow.keras.models import load_model

cascade_face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

model = load_model('C:\\Users\\Ashutosh Sharma\\Final\\Models\\model_v2.h5')

mixer.init()
sound = mixer.Sound('C:\\Users\\Ashutosh Sharma\\Final\\alarm.wav')

Score = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = cascade_face.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 150, 90), thickness=3)

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

            eye = roi_color[ey:ey+eh, ex:ex+ew]
            eye = cv2.resize(eye, (80, 80)) / 255.0
            eye = eye.reshape(80, 80, 3)
            eye = np.expand_dims(eye, axis=0)

            prediction = model.predict(eye)

            confidence = f'Confidence: {prediction[0][0]:.2f} (Closed) / {prediction[0][1]:.2f} (Open)'
            cv2.putText(frame, confidence, (10, height-40), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                        fontScale=0.6, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)

            if prediction[0][0] > 0.30:
                cv2.putText(frame, 'closed', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score ' + str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score += 1

                if Score > 5:
                    try:
                        sound.play()
                    except:
                        pass

            elif prediction[0][1] > 0.90:
                cv2.putText(frame, 'open', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score ' + str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX,
                            fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
                Score -= 1

                if Score < 0:
                    Score = 0

    cv2.imshow('Auto-Guard', frame)

    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()
